In [ ]:
# %% [markdown]
# ## 1. Configuration Initiale
# - Installation des dépendances
# - Import des bibliothèques
# - Configuration des widgets


import nest_asyncio
import time
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events
from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.core_plugins import ConversationSummaryPlugin


# Chargement des variables d'environnement
from dotenv import load_dotenv
load_dotenv()

# %% [markdown]
# ## 2. Interface Utilisateur
# - Zone de texte pour la description initiale
# - Bouton de démarrage

task_description = widgets.Textarea(
    value="Je veux créer un planning infirmier...",
    layout={'width': '90%', 'height': '100px'},
    description="Description:"
)

start_btn = widgets.Button(
    description="Démarrer la conversation",
    button_style='success',
    icon='rocket'
)

display(task_description)
display(start_btn)

# %% [markdown]
# ## 3. Paramètres de Planification
# - Structure de données pour les paramètres
# - Questions prédéfinies

class SchedulingParameters:
    def __init__(self):
        self.num_nurses = None
        self.num_days = None
        self.shifts = ['Matin', 'Après-midi', 'Nuit']
        self.constraints = {
            'max_consecutive_days': 5,
            'min_rest_days': 2
        }

required_params = [
    ("nombre d'infirmier(ère)s", "num_nurses", int),
    ("nombre de jours à planifier", "num_days", int),
    ("nombre maximum de jours consécutifs", "max_consecutive_days", int),
    ("jours de repos minimum par semaine", "min_rest_days", int)
]

# %% [markdown]
# ## 4. Agent Conversationnel
# - Initialisation du noyau Semantic Kernel
# - Configuration du modèle OpenAI

kernel = Kernel()
service = OpenAIChatCompletion(
    ai_model_id="gpt-4o",
    service_id="chat_completion",  # Add explicit service ID
    api_key="insert_key",
)
kernel.add_service(service)


async def generate_question(prompt):
    # Utilisation du nouveau format d'appel
    chat_history = ChatHistory()
    chat_history.add_user_message(prompt)
    
    response = await service.get_chat_message_contents(
        chat_history=chat_history,
        settings=service.get_prompt_execution_settings()
    )
    
    return str(response[0].content).strip('"')

async def collect_parameters():
    params = SchedulingParameters()
    
    for param_text, param_name, param_type in required_params:
        try:
            # Generate question using LLM
            question = await generate_question(
                f"Crée UNE SEULE question en français pour demander {param_text}. "
                f"Exemple: 'Combien d'infirmières ?' "
                f"Formule la question directement sans commentaires."
            )
            
            # Clear previous input
            input_box.value = ''
            
            # Display question
            display(widgets.HTML(f"<b>Assistant:</b> {question}"))
            
            # Get user input
            answer = await wait_for_user_input()
            setattr(params, param_name, param_type(answer))
            
        except Exception as e:
            display(widgets.HTML(f"<div style='color: red'>Erreur: {str(e)}</div>"))
            raise
    
    return params

# %% [markdown]
# ## 5. Gestion des Entrées Utilisateur
# - Mécanisme de capture des réponses

input_box = widgets.Text(description="Votre réponse:")
input_received = False

def on_submit(_):
    global input_received
    input_received = True

input_box.on_submit(on_submit)

async def wait_for_user_input():
    global input_received
    input_received = False
    display(input_box)
    
    with ui_events() as poll:
        while not input_received:
            poll(10)
            time.sleep(0.1)
    
    return input_box.value

# %% [markdown]
# ## 6. Intégration du Solveur
# - Chargement du solver local
# - Exécution du modèle CP-SAT

from importlib.machinery import SourceFileLoader
solver = SourceFileLoader("nurse_scheduler", "../src/nrp_or-tools.py").load_module()

def run_solver(params):
    scheduler = solver.NurseScheduler()
    return scheduler.generate_schedule(
        num_nurses=params.num_nurses,
        num_days=params.num_days,
        constraints=params.constraints
    )

# %% [markdown]
# ## 7. Flux Principal
# - Orchestration complète


async def main_flow():
    try:
        # Collecte initiale
        display(widgets.HTML("<h3>Début de la configuration...</h3>"))
        params = await collect_parameters()
        
        # Exécution du solver
        display(widgets.HTML("<h3>⏳ Calcul du planning...</h3>"))
        result = run_solver(params)
        
        # Affichage des résultats
        display(widgets.HTML(
            f"<h3>📅 Planning Généré</h3>"
            f"<pre>{result}</pre>"
        ))
    except Exception as e:
        display(widgets.HTML(f"<div style='color: red'>Error: {str(e)}</div>"))

def start_conversation(_):
    # Clear previous outputs
    display(widgets.HTML("<div id='output-container'></div>"))
    
    # Schedule async task properly
    import asyncio
    loop = asyncio.get_event_loop()
    loop.create_task(main_flow())

# Enable nested async in Jupyter
nest_asyncio.apply()
start_btn.on_click(start_conversation)


Textarea(value='Je veux créer un planning infirmier...', description='Description:', layout=Layout(height='100…

Button(button_style='success', description='Démarrer la conversation', icon='rocket', style=ButtonStyle())

/tmp/ipykernel_28368/2850172503.py:130: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_submit)


HTML(value="<div id='output-container'></div>")

HTML(value='<h3>Début de la configuration...</h3>')

HTML(value="<div style='color: red'>Erreur: 'OpenAIChatCompletion' object has no attribute 'get_prompt_executi…

HTML(value="<div style='color: red'>Error: 'OpenAIChatCompletion' object has no attribute 'get_prompt_executio…